In [ ]:
import pandas as pd
import json
from research.weight_estimation.akpd_utils.akpd_scorer import generate_confidence_score
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from research_lib.utils.data_access_utils import S3AccessUtils
from keras.models import load_model

In [ ]:
df = pd.read_csv('/root/data/alok/biomass_estimation/playground/depthable.csv')
anns = json.load(open('/root/data/alok/biomass_estimation/playground/depth_akpd.json'))
df['ann'] = anns

In [ ]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
s3 = S3AccessUtils('/root/data')
akpd_scorer_f, _, _ = s3.download_from_url(akpd_scorer_url)
akpd_scorer_network = load_model(akpd_scorer_f)

In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

In [ ]:
def compute_akpd_score(akpd_scorer_network, keypoints, camera_metadata) -> float:
    input_sample = {
        'keypoints': keypoints,
        'cm': camera_metadata,
        'stereo_pair_id': 0,
        'single_point_inference': True
    }

    akpd_score = generate_confidence_score(input_sample, akpd_scorer_network)
    return akpd_score


In [ ]:
weights, akpd_scores = [], []

count = 0
for idx, row in df.iterrows():
    ann = row.ann
    camera_metadata = json.loads(row.camera_metadata.replace("'", '"')) 
    akpd_score = compute_akpd_score(akpd_scorer_network, ann, camera_metadata)
    akpd_scores.append(akpd_score)
    
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )

    weight, _, _ = weight_estimator.predict(ann, cm)
    weights.append(weight)
    
    if count % 100 == 0:
        print(count)
    count += 1
    
    
    
    

In [ ]:
len(weights)

In [ ]:
df['akpd_score'] = akpd_scores

In [ ]:
df['weight'] = weights

In [ ]:
df.to_csv('/root/data/alok/biomass_estimation/playground/depthable_sealice_with_weights.csv')